In [61]:
from collections import Counter, defaultdict

In [62]:
# craete the corpus
with open("wiki_corpus.txt", "r", encoding="utf-8") as f:
    corpus = f.readlines() # list of strings


# define the vocab size:
# shoudl no be confused with the number of corpus new line.
vocab_size = 1000

In [63]:
word_freq = Counter()

In [64]:
# get stats: frequency of the words

word_freq = Counter()
corpus_words = " ".join(corpus).split()

for word in corpus_words:
    word_freq[word] += 1

In [65]:
# get the base vocab
# special characters
alphabet = []
for word in word_freq:
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)

alphabet.sort()

# add special token.
vocab = ["</w>"] + alphabet.copy()

In [66]:
# split each individual word into characters
splits = {word: [c for c in word] for word in word_freq}

In [ ]:
# merge the most frequent pair iteratively unitl the vocab size is reached
merges = {}

def generate_freq_pairs():
    freq_pairs = defaultdict(int)
    for word, freq in word_freq.items():
        charcters = splits[word]
        if len(charcters) == 1:
            continue
        for i in range(len(charcters) - 1):
            pair = (charcters[i], charcters[i+1])
            freq_pairs[pair] += freq
    return freq_pairs

def merge_pair(first_char, second_char):

    for word in word_freq:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) -1:
            if split[i] == first_char and split[i+1] == second_char:
                split = split[:i] + [first_char + second_char] + split[i+2:]
            else:
                i += 1
        splits[word] = split

    return splits


while len(vocab) < vocab_size:
    
    freq_pairs = generate_freq_pairs()

    best_pair = ("", "")
    max_freq = 0
    for pair, freq in freq_pairs.items():
        if freq > max_freq:
            best_pair = pair
            max_freq = freq

    # best_pair, max_freq = max(freq_pairs.items(), key=lambda x: x[1], default=(("", ""), 0))
    
    # merge the freq pair
    splits = merge_pair(*best_pair)
    merges[best_pair] =  best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])


def tokenize(text):
    words = [word for word in text.split()]
    splits = [[char for char in word] for word in words]

    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split
    
    return sum(splits, [])

    

tokenize("This is not a token.")
# best_pair = "ct"
# name = list("hector")
# name[:2] + [best_pair] + name[3+1:]

# did we token a space?

['This', 'is', 'not', 'a', 'to', 'k', 'en', '.']